In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
import os
import datetime
from tqdm import tqdm, trange
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep

# 1.下載資料: 臺北捷運每日分時各站OD流量統計資料

In [2]:
datalist = pd.read_csv('./Data/臺北捷運每日分時各站OD流量統計資料2.csv')
datalist.head()

,YYYYMM,Path
0,201901,http://tcgmetro.blob.core.windows.net/stationo...
1,201902,http://tcgmetro.blob.core.windows.net/stationo...
2,201903,http://tcgmetro.blob.core.windows.net/stationo...
3,201904,http://tcgmetro.blob.core.windows.net/stationo...
4,201905,http://tcgmetro.blob.core.windows.net/stationo...


In [3]:
datalist['Path'][0]

'http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201901.csv'

In [ ]:
已下載，不重複執行
for i in trange(len(datalist)):
    url = datalist['Path'][i]
    savepath = f'./Data/各站每日分時流量/{datalist.iat[i,0]}.csv'
    urlretrieve(url, savepath)

In [4]:
del datalist

# 2. 每日旅運量統計資料

資料來源
* 政府資料開放平台-資料集-臺北捷運全系統旅運量統計: https://data.gov.tw/dataset/128641
* 臺北大眾捷運-統計資料-旅運量: https://www.metro.taipei/cp.aspx?n=FF31501BEBDD0136

In [5]:
data = pd.read_csv('./Data/總旅運量/臺北捷運全系統旅運量統計_202106.csv', encoding='big5')
data.head()

,營運日,星期,總運量
0,110/6/1,?二?,"500,288"
1,110/6/2,?三?,"494,777"
2,110/6/3,?四?,"484,678"
3,110/6/4,?五?,"486,953"
4,110/6/5,?六?,"207,152"


In [6]:
def transferDate(string):
    y = int(string.split('/')[0])+1911
    m = int(string.split('/')[1])
    d = int(string.split('/')[2])
    return datetime.date(y, m, d)

In [7]:
def get_daily_ridership(df):
    date = df['營運日'].apply(lambda x: transferDate(x))
    quantity = df['總運量'].apply(lambda x: int(x.replace(',','')))
    return pd.DataFrame({'Date':date,'Ridership':quantity})

In [8]:
temp = get_daily_ridership(data)
temp.head()

,Date,Ridership
0,2021-06-01,500288
1,2021-06-02,494777
2,2021-06-03,484678
3,2021-06-04,486953
4,2021-06-05,207152


In [9]:
filelist = os.listdir('./Data/總旅運量/')
filelist[:5]

['臺北捷運全系統旅運量統計_201601.csv',
 '臺北捷運全系統旅運量統計_201602.csv',
 '臺北捷運全系統旅運量統計_201603.csv',
 '臺北捷運全系統旅運量統計_201604.csv',
 '臺北捷運全系統旅運量統計_201605.csv']

In [10]:
len(filelist)

66

In [11]:
ridership = pd.DataFrame()

for file in tqdm(filelist):
    data = pd.read_csv(f'./Data/總旅運量/{file}', encoding='big5')
    if len(ridership) == 0:
        ridership = get_daily_ridership(data)
    else:
        temp = get_daily_ridership(data)
        ridership = pd.concat([ridership, temp], ignore_index=True)

100%|██████████| 66/66 [00:00<00:00, 179.19it/s]


In [12]:
ridership.tail()

,Date,Ridership
2003,2021-06-26,278770
2004,2021-06-27,213596
2005,2021-06-28,593366
2006,2021-06-29,606268
2007,2021-06-30,620065


In [13]:
del file, filelist

In [14]:
ridership['Date'] = pd.to_datetime(ridership['Date'])
ridership['Year'] = ridership['Date'].dt.year
ridership['Month'] = ridership['Date'].dt.month
ridership['DayofWeek'] = ridership['Date'].dt.dayofweek
ridership = ridership[['Date','Year','Month','DayofWeek','Ridership']]
ridership.head()

,Date,Year,Month,DayofWeek,Ridership
0,2016-01-01,2016,1,4,1873454
1,2016-01-02,2016,1,5,1846862
2,2016-01-03,2016,1,6,1507044
3,2016-01-04,2016,1,0,1991063
4,2016-01-05,2016,1,1,2044627


In [15]:
#存檔
ridership.to_csv('./Data/Taipei_Metro_Ridership_201601-202106.csv', index=False)

In [16]:
del ridership

# 3. 各站每日進出次數資料

資料來源
* 政府資料開放平台-資料集-臺北捷運各站進出量統計: https://data.gov.tw/dataset/128683
* 臺北大眾捷運-統計資料-各站進出量統計: https://www.metro.taipei/cp.aspx?n=FF31501BEBDD0136

In [17]:
def get_daily_entrance_exit(filename):
    filepath=f'./Data/各站日進出量/{filename}'
    df_exit = pd.read_excel(filepath, engine='odf', sheet_name='出站資料')
    df_exit = pd.melt(df_exit, id_vars=df_exit.columns[0], var_name='Station', value_name='Exit_Counts')
    df_exit.rename(columns={df_exit.columns[0]:'Date'}, inplace=True)
    df_entrance = pd.read_excel(filepath, engine='odf', sheet_name='進站資料')
    df_entrance = pd.melt(df_entrance, id_vars=df_entrance.columns[0],var_name='Station',value_name='Entrance_Counts')
    df_entrance.rename(columns={df_entrance.columns[0]:'Date'},inplace=True)
    if len(df_exit) == len(df_entrance):
        df = df_exit.merge(df_entrance, how='inner', on=['Date', 'Station'])
    else:
        df = None
        print(filename)
        print(f'出站資料共{len(df_exit)}筆，進站資料共{len(df_entrance)}筆，筆數不相符。')

    return df

In [18]:
get_daily_entrance_exit('202106_cht.ods')

,Date,Station,Exit_Counts,Entrance_Counts
0,2021-06-01,松山機場,963,1010
1,2021-06-02,松山機場,954,947
2,2021-06-03,松山機場,879,874
3,2021-06-04,松山機場,985,1009
4,2021-06-05,松山機場,335,358
...,...,...,...,...
3565,2021-06-26,新北產業園區,604,623
3566,2021-06-27,新北產業園區,495,437
3567,2021-06-28,新北產業園區,1626,1667
3568,2021-06-29,新北產業園區,1695,1721


In [19]:
filelist = os.listdir('./Data/各站日進出量/')
len(filelist)

66

In [20]:
filelist[:3], filelist[-3:]

(['201601_cht.ods', '201602_cht.ods', '201603_cht.ods'],
 ['202104_cht.ods', '202105_cht.ods', '202106_cht.ods'])

In [21]:
station_ridership = pd.DataFrame()

for file in tqdm(filelist):
    if len(station_ridership) == 0:
        station_ridership = get_daily_entrance_exit(file)
    else:
        temp = get_daily_entrance_exit(file)
        station_ridership = pd.concat([station_ridership, temp], ignore_index=True)

100%|██████████| 66/66 [01:22<00:00,  1.25s/it]


In [22]:
station_ridership.head()

,Date,Station,Exit_Counts,Entrance_Counts
0,2016-01-01,松山機場,5343,4818
1,2016-01-02,松山機場,4387,4473
2,2016-01-03,松山機場,4750,5653
3,2016-01-04,松山機場,5599,5592
4,2016-01-05,松山機場,5542,5417


In [23]:
station_ridership.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222881 entries, 0 to 222880
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             222881 non-null  datetime64[ns]
 1   Station          222881 non-null  object        
 2   Exit_Counts      222881 non-null  int64         
 3   Entrance_Counts  222881 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 6.8+ MB


### 新增年、月、星期、加總進出站

In [24]:
station_ridership['Year'] = station_ridership['Date'].dt.year
station_ridership['Month'] = station_ridership['Date'].dt.month
station_ridership['DayofWeek'] = station_ridership['Date'].dt.dayofweek
station_ridership['Counts'] = station_ridership['Exit_Counts'] + station_ridership['Entrance_Counts']
station_ridership.tail()

,Date,Station,Exit_Counts,Entrance_Counts,Year,Month,DayofWeek,Counts
222876,2021-06-26,新北產業園區,604,623,2021,6,5,1227
222877,2021-06-27,新北產業園區,495,437,2021,6,6,932
222878,2021-06-28,新北產業園區,1626,1667,2021,6,0,3293
222879,2021-06-29,新北產業園區,1695,1721,2021,6,1,3416
222880,2021-06-30,新北產業園區,1714,1685,2021,6,2,3399


### 2020年新增環狀線，原「板橋」站改為「BL板橋」，故更正2016-2019「板橋」站改為與2020年相符

In [25]:
station_ridership['Station'].unique()

array(['松山機場', '中山國中', '南京復興', '忠孝復興', '大安', '科技大樓', '六張犁', '麟光', '辛亥',
       '萬芳醫院', '萬芳社區', '木柵', '動物園', '大直', '劍南路', '西湖', '港墘', '文德', '內湖',
       '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '小碧潭', '新店', '新店區公所', '七張',
       '大坪林', '景美', '萬隆', '公館', '台電大樓', '古亭', '中正紀念堂', '小南門', '頂溪',
       '永安市場', '景安', '南勢角', '台大醫院', '台北車站', '中山', '雙連', '民權西路', '圓山',
       '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投', '新北投', '復興崗',
       '忠義', '關渡', '竹圍', '紅樹林', '淡水', '頂埔', '永寧', '土城', '海山', '亞東醫院',
       '府中', '板橋', '新埔', '江子翠', '龍山寺', '西門', '善導寺', '忠孝新生', '忠孝敦化',
       '國父紀念館', '市政府', '永春', '後山埤', '昆陽', '南港', '象山', '台北101/世貿', '信義安和',
       '大安森林公園', '北門', '松江南京', '台北小巨蛋', '南京三民', '松山', '輔大', '新莊', '頭前庄',
       '先嗇宮', '三重', '菜寮', '台北橋', '大橋頭', '中山國小', '行天宮', '東門', '蘆洲', '三民高中',
       '徐匯中學', '三和國中', '三重國小', '迴龍', '丹鳳', 'BL板橋', '十四張', '秀朗橋', '景平',
       '中和', '橋和', '中原', '板新', 'Y板橋', '新埔民生', '幸福', '新北產業園區'],
      dtype=object)

In [26]:
station_ridership[station_ridership['Station'] == '板橋'].shape[0]

1461

In [27]:
station_ridership[station_ridership['Station'] == '板橋']['Date'].min()

Timestamp('2016-01-01 00:00:00')

In [28]:
station_ridership[station_ridership['Station'] == '板橋']['Date'].max()

Timestamp('2019-12-31 00:00:00')

In [29]:
for i in station_ridership[station_ridership['Station'] == '板橋'].index:
    station_ridership.at[i,'Station'] = 'BL板橋'

In [30]:
station_ridership[station_ridership['Station'] == '板橋']

,Date,Station,Exit_Counts,Entrance_Counts,Year,Month,DayofWeek,Counts


In [31]:
station_ridership['Station'].unique()

array(['松山機場', '中山國中', '南京復興', '忠孝復興', '大安', '科技大樓', '六張犁', '麟光', '辛亥',
       '萬芳醫院', '萬芳社區', '木柵', '動物園', '大直', '劍南路', '西湖', '港墘', '文德', '內湖',
       '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '小碧潭', '新店', '新店區公所', '七張',
       '大坪林', '景美', '萬隆', '公館', '台電大樓', '古亭', '中正紀念堂', '小南門', '頂溪',
       '永安市場', '景安', '南勢角', '台大醫院', '台北車站', '中山', '雙連', '民權西路', '圓山',
       '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投', '新北投', '復興崗',
       '忠義', '關渡', '竹圍', '紅樹林', '淡水', '頂埔', '永寧', '土城', '海山', '亞東醫院',
       '府中', 'BL板橋', '新埔', '江子翠', '龍山寺', '西門', '善導寺', '忠孝新生', '忠孝敦化',
       '國父紀念館', '市政府', '永春', '後山埤', '昆陽', '南港', '象山', '台北101/世貿', '信義安和',
       '大安森林公園', '北門', '松江南京', '台北小巨蛋', '南京三民', '松山', '輔大', '新莊', '頭前庄',
       '先嗇宮', '三重', '菜寮', '台北橋', '大橋頭', '中山國小', '行天宮', '東門', '蘆洲', '三民高中',
       '徐匯中學', '三和國中', '三重國小', '迴龍', '丹鳳', '十四張', '秀朗橋', '景平', '中和', '橋和',
       '中原', '板新', 'Y板橋', '新埔民生', '幸福', '新北產業園區'], dtype=object)

In [32]:
station_ridership.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222881 entries, 0 to 222880
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             222881 non-null  datetime64[ns]
 1   Station          222881 non-null  object        
 2   Exit_Counts      222881 non-null  int64         
 3   Entrance_Counts  222881 non-null  int64         
 4   Year             222881 non-null  int64         
 5   Month            222881 non-null  int64         
 6   DayofWeek        222881 non-null  int64         
 7   Counts           222881 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 13.6+ MB


In [33]:
station_ridership = station_ridership[['Date','Year','Month','DayofWeek','Station','Entrance_Counts','Exit_Counts','Counts']]
station_ridership.tail()

,Date,Year,Month,DayofWeek,Station,Entrance_Counts,Exit_Counts,Counts
222876,2021-06-26,2021,6,5,新北產業園區,623,604,1227
222877,2021-06-27,2021,6,6,新北產業園區,437,495,932
222878,2021-06-28,2021,6,0,新北產業園區,1667,1626,3293
222879,2021-06-29,2021,6,1,新北產業園區,1721,1695,3416
222880,2021-06-30,2021,6,2,新北產業園區,1685,1714,3399


In [34]:
#Dataset存檔
station_ridership.to_csv('./Data/Taipei_Metro_Ridership_by_Station_201601-202106.csv', index=False)

In [35]:
del file, filelist, station_ridership

# 4.各站每日分時OD流量統計資料

#### 每個路線每日流量: 201901-202106

In [47]:
def get_daily_routes_counts(filename):
    path = f'./Data/各站每日分時流量/{filename}'
    data = pd.read_csv(path)
    data.rename(columns={'日期':'Date','時段':'Hour','進站':'Entrance','出站':'Exit','人次':'Counts'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'] )
    df = pd.DataFrame(data.groupby(['Date','Entrance','Exit'])['Counts'].sum()).reset_index()
    return df

In [48]:
get_daily_routes_counts('201901.csv')

,Date,Entrance,Exit,Counts
0,2019-01-01,BL板橋,BL板橋,320
1,2019-01-01,BL板橋,七張,184
2,2019-01-01,BL板橋,三和國中,153
3,2019-01-01,BL板橋,三民高中,132
4,2019-01-01,BL板橋,三重,23
...,...,...,...,...
361579,2019-01-31,龍山寺,頂埔,272
361580,2019-01-31,龍山寺,頂溪,322
361581,2019-01-31,龍山寺,頭前庄,20
361582,2019-01-31,龍山寺,麟光,53


In [2]:
filelist = os.listdir('./Data/各站每日分時流量/')
filelist[:3], filelist[-3:]

(['201901.csv', '201902.csv', '201903.csv'],
 ['202104.csv', '202105.csv', '202106.csv'])

In [3]:
len(filelist)

30

In [51]:
daily_routes = pd.DataFrame()

for file in tqdm(filelist):
    if len(daily_routes) == 0:
        daily_routes = get_daily_routes_counts(file)
    else:
        temp = get_daily_routes_counts(file)
        daily_routes = pd.concat([daily_routes, temp], ignore_index=True)

100%|██████████| 30/30 [03:21<00:00,  6.73s/it]


In [52]:
daily_routes.tail()

,Date,Entrance,Exit,Counts
12029247,2021-06-30,龍山寺,頂埔,62
12029248,2021-06-30,龍山寺,頂溪,49
12029249,2021-06-30,龍山寺,頭前庄,4
12029250,2021-06-30,龍山寺,麟光,3
12029251,2021-06-30,龍山寺,龍山寺,61


In [53]:
daily_routes.to_csv('./Data/Taipei_Metro_Ridership_by_Route_201901-202106.csv', index=False)

#### 合併各站分時流量

In [6]:
routes_counts = pd.DataFrame()

for file in tqdm(filelist):
    data = pd.read_csv(f'./Data/各站每日分時流量/{file}')
    if len(routes_counts) == 0:
        routes_counts = data
    else:
        routes_counts = pd.concat([routes_counts, data], ignore_index=True)

100%|██████████| 30/30 [07:18<00:00, 14.61s/it]


In [7]:
routes_counts.tail()

,日期,時段,進站,出站,人次
252135276,2021-06-30,23,新北產業園區,板新,1
252135277,2021-06-30,23,新北產業園區,Y板橋,2
252135278,2021-06-30,23,新北產業園區,新埔民生,1
252135279,2021-06-30,23,新北產業園區,幸福,1
252135280,2021-06-30,23,新北產業園區,新北產業園區,0


In [8]:
routes_counts.rename(columns={'日期':'Date','時段':'Hour','進站':'Entrance','出站':'Exit','人次':'Counts'}, inplace=True)
routes_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252135281 entries, 0 to 252135280
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Date      object
 1   Hour      int64 
 2   Entrance  object
 3   Exit      object
 4   Counts    int64 
dtypes: int64(2), object(3)
memory usage: 9.4+ GB


「大橋頭」名稱與其他檔案不相符，這邊使用「大橋頭站」，修改以方便比對

In [9]:
len(routes_counts[routes_counts['Entrance'] == '大橋頭站']), len(routes_counts[routes_counts['Exit'] == '大橋頭站'])

(2195335, 2195335)

In [11]:
for i in routes_counts[routes_counts['Entrance'] == '大橋頭站'].index:
    routes_counts.at[i,'Entrance'] = '大橋頭'
    
for i in routes_counts[routes_counts['Exit'] == '大橋頭站'].index:
    routes_counts.at[i,'Exit'] = '大橋頭'

In [12]:
len(routes_counts[routes_counts['Entrance'] == '大橋頭站']), len(routes_counts[routes_counts['Exit'] == '大橋頭站'])

(0, 0)

In [13]:
len(routes_counts[routes_counts['Entrance'] == '板橋']), len(routes_counts[routes_counts['Exit'] == '板橋'])

(0, 0)

In [14]:
routes_counts.tail()

,Date,Hour,Entrance,Exit,Counts
252135276,2021-06-30,23,新北產業園區,板新,1
252135277,2021-06-30,23,新北產業園區,Y板橋,2
252135278,2021-06-30,23,新北產業園區,新埔民生,1
252135279,2021-06-30,23,新北產業園區,幸福,1
252135280,2021-06-30,23,新北產業園區,新北產業園區,0


In [15]:
routes_counts.to_csv('./Data/Taipei_Metro_Route_hourly_2019-202106.csv', index=False)

# 5. 每個站到站的行駛時間

#### 爬蟲

In [3]:
driver_path = 'C:/Users/angel/geckodriver.exe'
driver = webdriver.Firefox(executable_path=driver_path)
driver.get('https://web.metro.taipei/pages/tw/ticketroutetimesingle/019')

In [4]:
select = Select(driver.find_element_by_css_selector('#menulistsingle'))
len(select.options)

136

In [5]:
select.options[0].text, select.options[1].text, select.options[135].text

('請選擇車站：', 'BR01 動物園', 'Y20 新北產業園區')

In [6]:
stationA, stationB, traveltime = [],[],[]

In [7]:
elem = driver.find_elements_by_xpath('//td')
elem[0].text,elem[2].text,elem[7].text

('BR01 動物園', 'BR02 木柵', '2')

In [8]:
for i in range(0,len(elem),8):
    stationA.append(elem[i].text)
for i in range(2,len(elem),8):
    stationB.append(elem[i].text)
for i in range(7,len(elem),8):
    traveltime.append(elem[i].text)

In [9]:
len(stationA), len(stationB), len(traveltime)

(134, 134, 134)

In [11]:
len(stationA) == len(stationB) == len(traveltime)

True

In [10]:
driver.current_url

'https://web.metro.taipei/pages/tw/ticketroutetimesingle/019'

In [12]:
for i in range(2,len(select.options)):
    url = driver.current_url
    select = Select(driver.find_element_by_css_selector('#menulistsingle'))
    select.select_by_index(i)
    driver.find_element_by_xpath('/html/body/app-root/app-ticketroutetimesingle/section[1]/div/div[2]/button').click()
    sleep(10)
    if url != driver.current_url:
        print(f'{select.options[i].text}')
        elem = driver.find_elements_by_xpath('//td')
        for _ in range(0,len(elem),8):
            stationA.append(elem[_].text)
        for _ in range(2,len(elem),8):
            stationB.append(elem[_].text)
        for _ in range(7,len(elem),8):
            traveltime.append(elem[_].text)
        
        if len(stationA) == len(stationB) == len(traveltime):
            continue
        else:
            print(f'{driver.current_url}')
            print(f'{select.options[i].text}')
            break
    else:
        print(f'原url:{url}, 現在url:{driver.current_url}')
        print(f'{select.options[i].text}')
        break

BR02 木柵
BR03 萬芳社區
BR04 萬芳醫院
BR05 辛亥
BR06 麟光
BR07 六張犁
BR08 科技大樓
BR09 大安
BR10 忠孝復興
BR11 南京復興
BR12 中山國中
BR13 松山機場
BR14 大直
BR15 劍南路
BR16 西湖
BR17 港墘
BR18 文德
BR19 內湖
BR20 大湖公園
BR21 葫洲
BR22 東湖
BR23 南港軟體園區
BR24 南港展覽館
R02 象山
R03 台北101/世貿
R04 信義安和
R05 大安
R06 大安森林公園
R07 東門
R08 中正紀念堂
R09 台大醫院
R10 台北車站
R11 中山
R12 雙連
R13 民權西路
R14 圓山
R15 劍潭
R16 士林
R17 芝山
R18 明德
R19 石牌
R20 唭哩岸
R21 奇岩
R22 北投
R22A 新北投
R23 復興崗
R24 忠義
R25 關渡
R26 竹圍
R27 紅樹林
R28 淡水
G01 新店
G02 新店區公所
G03 七張
G03A 小碧潭
G04 大坪林
G05 景美
G06 萬隆
G07 公館
G08 台電大樓
G09 古亭
G10 中正紀念堂
G11 小南門
G12 西門
G13 北門
G14 中山
G15 松江南京
G16 南京復興
G17 台北小巨蛋
G18 南京三民
G19 松山
O01 南勢角
O02 景安
O03 永安市場
O04 頂溪
O05 古亭
O06 東門
O07 忠孝新生
O08 松江南京
O09 行天宮
O10 中山國小
O11 民權西路
O12 大橋頭
O13 台北橋
O14 菜寮
O15 三重
O16 先嗇宮
O17 頭前庄
O18 新莊
O19 輔大
O20 丹鳳
O21 迴龍
O50 三重國小
O51 三和國中
O52 徐匯中學
O53 三民高中
O54 蘆洲
BL01 頂埔
BL02 永寧
BL03 土城
BL04 海山
BL05 亞東醫院
BL06 府中
BL07 板橋
BL08 新埔
BL09 江子翠
BL10 龍山寺
BL11 西門
BL12 台北車站
BL13 善導寺
BL14 忠孝新生
BL15 忠孝復興
BL16 忠孝敦化
BL17 國父紀念館
BL18 市政府
BL19 永春
BL20 後山埤
BL21 昆陽
BL22 南港
BL23 南港展

In [13]:
len(stationA), len(stationB), len(traveltime)

(18058, 18058, 18058)

In [16]:
driver.close()

In [33]:
len(stationA), len(stationB), len(traveltime)

(18058, 18058, 18058)

In [36]:
traveltime = pd.DataFrame({'From':stationA, 'To':stationB, 'TravelTime':traveltime})
traveltime.head()

,From,To,TravelTime
0,BR01 動物園,BR02 木柵,2
1,BR01 動物園,BR03 萬芳社區,3
2,BR01 動物園,BR04 萬芳醫院,5
3,BR01 動物園,BR05 辛亥,7
4,BR01 動物園,BR06 麟光,10


In [43]:
traveltime['StationA'] = traveltime['From'].apply(lambda _: _.split(' ')[-1])
traveltime['StationB'] = traveltime['To'].apply(lambda _: _.split(' ')[-1])
traveltime.head()

,From,To,TravelTime,StationA,StationB
0,BR01 動物園,BR02 木柵,2,動物園,木柵
1,BR01 動物園,BR03 萬芳社區,3,動物園,萬芳社區
2,BR01 動物園,BR04 萬芳醫院,5,動物園,萬芳醫院
3,BR01 動物園,BR05 辛亥,7,動物園,辛亥
4,BR01 動物園,BR06 麟光,10,動物園,麟光


In [44]:
traveltime['StationA'].unique()

array(['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安',
       '忠孝復興', '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德',
       '內湖', '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '象山', '台北101/世貿',
       '信義安和', '大安森林公園', '東門', '中正紀念堂', '台大醫院', '台北車站', '中山', '雙連',
       '民權西路', '圓山', '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投',
       '新北投', '復興崗', '忠義', '關渡', '竹圍', '紅樹林', '淡水', '新店', '新店區公所', '七張',
       '小碧潭', '大坪林', '景美', '萬隆', '公館', '台電大樓', '古亭', '小南門', '西門', '北門',
       '松江南京', '台北小巨蛋', '南京三民', '松山', '南勢角', '景安', '永安市場', '頂溪', '忠孝新生',
       '行天宮', '中山國小', '大橋頭', '台北橋', '菜寮', '三重', '先嗇宮', '頭前庄', '新莊', '輔大',
       '丹鳳', '迴龍', '三重國小', '三和國中', '徐匯中學', '三民高中', '蘆洲', '頂埔', '永寧', '土城',
       '海山', '亞東醫院', '府中', '板橋', '新埔', '江子翠', '龍山寺', '善導寺', '忠孝敦化',
       '國父紀念館', '市政府', '永春', '後山埤', '昆陽', '南港', '十四張', '秀朗橋', '景平', '中和',
       '橋和', '中原', '板新', '新埔民生', '幸福', '新北產業園區'], dtype=object)

In [45]:
traveltime['StationB'].unique()

array(['木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興',
       '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德', '內湖',
       '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '象山', '台北101/世貿', '信義安和',
       '大安森林公園', '東門', '中正紀念堂', '台大醫院', '台北車站', '中山', '雙連', '民權西路', '圓山',
       '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投', '新北投', '復興崗',
       '忠義', '關渡', '竹圍', '紅樹林', '淡水', '新店', '新店區公所', '七張', '小碧潭', '大坪林',
       '景美', '萬隆', '公館', '台電大樓', '古亭', '小南門', '西門', '北門', '松江南京', '台北小巨蛋',
       '南京三民', '松山', '南勢角', '景安', '永安市場', '頂溪', '忠孝新生', '行天宮', '中山國小',
       '大橋頭', '台北橋', '菜寮', '三重', '先嗇宮', '頭前庄', '新莊', '輔大', '丹鳳', '迴龍',
       '三重國小', '三和國中', '徐匯中學', '三民高中', '蘆洲', '頂埔', '永寧', '土城', '海山',
       '亞東醫院', '府中', '板橋', '新埔', '江子翠', '龍山寺', '善導寺', '忠孝敦化', '國父紀念館',
       '市政府', '永春', '後山埤', '昆陽', '南港', '十四張', '秀朗橋', '景平', '中和', '橋和',
       '中原', '板新', '新埔民生', '幸福', '新北產業園區', '動物園'], dtype=object)

In [50]:
traveltime['Code_StationA'] = traveltime['StationA'].map(stations_map)
traveltime['Code_StationB'] = traveltime['StationB'].map(stations_map)
traveltime.head()

,From,To,TravelTime,StationA,StationB,Code_StationA,Code_StationB
0,BR01 動物園,BR02 木柵,2,動物園,木柵,BR01,BR02
1,BR01 動物園,BR03 萬芳社區,3,動物園,萬芳社區,BR01,BR03
2,BR01 動物園,BR04 萬芳醫院,5,動物園,萬芳醫院,BR01,BR04
3,BR01 動物園,BR05 辛亥,7,動物園,辛亥,BR01,BR05
4,BR01 動物園,BR06 麟光,10,動物園,麟光,BR01,BR06


#### 板橋沒有分BL板橋與Y板橋，手動修改StationB，以配合路線計算

In [66]:
stations_map['Y板橋']

'Y16'

In [60]:
for i in traveltime[(traveltime['StationA'] == '板橋') & (traveltime['From'] == 'BL07 板橋')].index:
    traveltime.at[i,'Code_StationA'] = stations_map['BL板橋']

In [67]:
for i in traveltime[(traveltime['StationA'] == '板橋') & (traveltime['From'] == 'Y16 板橋')].index:
    traveltime.at[i,'Code_StationA'] = stations_map['Y板橋']

In [71]:
for i in traveltime[(traveltime['StationB'] == '板橋') & (traveltime['To'] == 'BL07 板橋')].index:
    traveltime.at[i,'Code_StationB'] = stations_map['BL板橋']

In [73]:
for i in traveltime[(traveltime['StationB'] == '板橋') & (traveltime['To'] == 'Y16 板橋')].index:
    traveltime.at[i,'Code_StationB'] = stations_map['Y板橋']

In [74]:
traveltime[traveltime['StationB'] == '板橋']

,From,To,TravelTime,StationA,StationB,Code_StationA,Code_StationB
103,BR01 動物園,BL07 板橋,40,動物園,板橋,BR01,BL07
129,BR01 動物園,Y16 板橋,51,動物園,板橋,BR01,Y16
237,BR02 木柵,BL07 板橋,38,木柵,板橋,BR02,BL07
263,BR02 木柵,Y16 板橋,49,木柵,板橋,BR02,Y16
371,BR03 萬芳社區,BL07 板橋,37,萬芳社區,板橋,BR03,BL07
...,...,...,...,...,...,...,...
17786,Y18 O17 頭前庄,Y16 板橋,6,頭前庄,板橋,O17,Y16
17894,Y19 幸福,BL07 板橋,20,幸福,板橋,Y19,BL07
17920,Y19 幸福,Y16 板橋,9,幸福,板橋,Y19,Y16
18028,Y20 新北產業園區,BL07 板橋,22,新北產業園區,板橋,Y20,BL07


In [75]:
traveltime['From_To'] = traveltime['Code_StationA'] + '_' + traveltime['Code_StationB']
traveltime

,From,To,TravelTime,StationA,StationB,Code_StationA,Code_StationB,From_To
0,BR01 動物園,BR02 木柵,2,動物園,木柵,BR01,BR02,BR01_BR02
1,BR01 動物園,BR03 萬芳社區,3,動物園,萬芳社區,BR01,BR03,BR01_BR03
2,BR01 動物園,BR04 萬芳醫院,5,動物園,萬芳醫院,BR01,BR04,BR01_BR04
3,BR01 動物園,BR05 辛亥,7,動物園,辛亥,BR01,BR05,BR01_BR05
4,BR01 動物園,BR06 麟光,10,動物園,麟光,BR01,BR06,BR01_BR06
...,...,...,...,...,...,...,...,...
18053,Y20 新北產業園區,Y15 板新,14,新北產業園區,板新,Y20,Y15,Y20_Y15
18054,Y20 新北產業園區,Y16 板橋,11,新北產業園區,板橋,Y20,Y16,Y20_Y16
18055,Y20 新北產業園區,Y17 新埔民生,8,新北產業園區,新埔民生,Y20,Y17,Y20_Y17
18056,Y20 新北產業園區,Y18 頭前庄,5,新北產業園區,頭前庄,Y20,O17,Y20_O17


In [76]:
traveltime = traveltime[['StationA','StationB','Code_StationA','Code_StationB','From_To','TravelTime']]
traveltime.tail()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
18053,新北產業園區,板新,Y20,Y15,Y20_Y15,14
18054,新北產業園區,板橋,Y20,Y16,Y20_Y16,11
18055,新北產業園區,新埔民生,Y20,Y17,Y20_Y17,8
18056,新北產業園區,頭前庄,Y20,O17,Y20_O17,5
18057,新北產業園區,幸福,Y20,Y19,Y20_Y19,3


In [77]:
traveltime.to_csv('./Data/Taipei_Metro_Route_Traveltime.csv', index=False, encoding='UTF-8')

#### 補清資料：板橋站、重複路線

In [49]:
traveltime = pd.read_csv('./Data/Taipei_Metro_Route_Traveltime.csv', encoding='UTF-8')
traveltime.tail()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
18053,新北產業園區,板新,Y20,Y15,Y20_Y15,14
18054,新北產業園區,板橋,Y20,Y16,Y20_Y16,11
18055,新北產業園區,新埔民生,Y20,Y17,Y20_Y17,8
18056,新北產業園區,頭前庄,Y20,O17,Y20_O17,5
18057,新北產業園區,幸福,Y20,Y19,Y20_Y19,3


In [50]:
traveltime[traveltime['Code_StationA'] == 'BL07'].head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
13912,板橋,動物園,BL07,BR01,BL07_BR01,40
13913,板橋,木柵,BL07,BR02,BL07_BR02,38
13914,板橋,萬芳社區,BL07,BR03,BL07_BR03,37
13915,板橋,萬芳醫院,BL07,BR04,BL07_BR04,35
13916,板橋,辛亥,BL07,BR05,BL07_BR05,33


In [51]:
traveltime[traveltime['Code_StationA'] == 'Y16'].head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
17389,板橋,動物園,Y16,BR01,Y16_BR01,51
17390,板橋,木柵,Y16,BR02,Y16_BR02,49
17391,板橋,萬芳社區,Y16,BR03,Y16_BR03,48
17392,板橋,萬芳醫院,Y16,BR04,Y16_BR04,46
17393,板橋,辛亥,Y16,BR05,Y16_BR05,44


In [52]:
for i in traveltime[traveltime['Code_StationA'] == 'BL07'].index:
    traveltime.at[i,'StationA'] = 'BL板橋'
    
for i in traveltime[traveltime['Code_StationB'] == 'BL07'].index:
    traveltime.at[i,'StationB'] = 'BL板橋'
    
for i in traveltime[traveltime['Code_StationA'] == 'Y16'].index:
    traveltime.at[i,'StationA'] = 'Y板橋'
    
for i in traveltime[traveltime['Code_StationB'] == 'Y16'].index:
    traveltime.at[i,'StationB'] = 'Y板橋'

In [53]:
traveltime[traveltime['Code_StationA'] == 'BL07'].head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
13912,BL板橋,動物園,BL07,BR01,BL07_BR01,40
13913,BL板橋,木柵,BL07,BR02,BL07_BR02,38
13914,BL板橋,萬芳社區,BL07,BR03,BL07_BR03,37
13915,BL板橋,萬芳醫院,BL07,BR04,BL07_BR04,35
13916,BL板橋,辛亥,BL07,BR05,BL07_BR05,33


In [54]:
traveltime[traveltime['Code_StationA'] == 'Y16'].head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
17389,Y板橋,動物園,Y16,BR01,Y16_BR01,51
17390,Y板橋,木柵,Y16,BR02,Y16_BR02,49
17391,Y板橋,萬芳社區,Y16,BR03,Y16_BR03,48
17392,Y板橋,萬芳醫院,Y16,BR04,Y16_BR04,46
17393,Y板橋,辛亥,Y16,BR05,Y16_BR05,44


In [55]:
traveltime[traveltime.duplicated()]

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
26,動物園,大安,BR01,BR09,BR01_BR09,15
61,動物園,中正紀念堂,BR01,G10,BR01_G10,27
65,動物園,中山,BR01,G14,BR01_G14,28
67,動物園,南京復興,BR01,BR11,BR01_BR11,19
75,動物園,古亭,BR01,G09,BR01_G09,29
...,...,...,...,...,...,...
18036,新北產業園區,忠孝復興,Y20,BL15,Y20_BL15,32
18044,新北產業園區,南港展覽館,Y20,BL23,Y20_BL23,48
18045,新北產業園區,大坪林,Y20,G04,Y20_G04,33
18049,新北產業園區,景安,Y20,O02,Y20_O02,24


In [56]:
traveltime[traveltime['From_To'] == 'BR01_BR09'] 

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
7,動物園,大安,BR01,BR09,BR01_BR09,15
26,動物園,大安,BR01,BR09,BR01_BR09,15


In [57]:
traveltime = traveltime.drop_duplicates().reset_index(drop=True)
traveltime.tail()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
14037,新北產業園區,中原,Y20,Y14,Y20_Y14,17
14038,新北產業園區,板新,Y20,Y15,Y20_Y15,14
14039,新北產業園區,Y板橋,Y20,Y16,Y20_Y16,11
14040,新北產業園區,新埔民生,Y20,Y17,Y20_Y17,8
14041,新北產業園區,幸福,Y20,Y19,Y20_Y19,3


In [60]:
len(traveltime['From_To'].unique())

14042

In [61]:
traveltime.to_csv('./Data/Taipei_Metro_Route_Traveltime_v2.csv', index=False, encoding='UTF-8')

# 6.捷運站資料

In [2]:
stations = pd.read_csv('./Data/Taipei_Metro_Stations.csv')
stations.head()

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation
0,BL01,頂埔,BL,N,0,0,0
1,BL02,永寧,BL,N,0,0,0
2,BL03,土城,BL,N,0,0,0
3,BL04,海山,BL,N,0,0,0
4,BL05,亞東醫院,BL,N,0,0,0


In [3]:
geodata = pd.read_csv('./Data/northern-taiwan.csv')
geodata.head()

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
0,BR01,BR13,動物園,Taipei Zoo,BR,文湖線,台北市文山區新光路二段32號,24.998197,121.579338
1,BR02,BR12,木柵,Muzha,BR,文湖線,台北市文山區木柵路四段135號,24.998241,121.573145
2,BR03,BR11,萬芳社區,Wanfang Community,BR,文湖線,台北市文山區萬芳路60號,24.998585,121.568102
3,BR04,BR10,萬芳醫院,Wanfang Hospital,BR,文湖線,台北市文山區興隆路三段113號,24.999386,121.558152
4,BR05,BR9,辛亥,Xinhai,BR,文湖線,台北市文山區辛亥路四段128號,25.005475,121.557107


#### 經緯度資料，板橋站都用「板橋」，為方便對應，將板南線板橋站改為BL板橋，環狀線板橋改為Y板橋

In [4]:
s1 = set(stations['Station'].unique())
s2 = set(geodata['station_name_tw'].unique())
s1.difference(s2)

{'BL板橋', 'Y板橋'}

In [5]:
geodata[geodata['station_name_tw'] == '板橋']

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
104,BL07,BL2,板橋,Banqiao,BL,板南線,新北市板橋區站前路5號B1,25.013618,121.462302
130,Y16,Y15,板橋,Banqiao,Y,環狀線,新北市板橋區新站路66號,25.015156,121.464825


In [6]:
geodata.at[104, 'station_name_tw'] = 'BL板橋'
geodata.at[130, 'station_name_tw'] = 'Y板橋'
geodata[geodata['station_name_en'] == 'Banqiao']

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
104,BL07,BL2,BL板橋,Banqiao,BL,板南線,新北市板橋區站前路5號B1,25.013618,121.462302
130,Y16,Y15,Y板橋,Banqiao,Y,環狀線,新北市板橋區新站路66號,25.015156,121.464825


#### 經緯度資料若只用'station_name_tw'對應會重複，如轉乘站，以捷運站名稱、經度、緯度為準去除重複值

In [7]:
geodata[geodata['station_name_tw'].duplicated()].head()

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
27,R05,R8,大安,Daan,R,淡水信義線,台北市大安區信義路四段2號,25.032943,121.543551
62,G10,G11,中正紀念堂,Chiang Kai-Shek Memorial Hall,G,松山新店線,台北市中正區羅斯福路一段8之1號B1,25.032729,121.518270
66,G14,G16,中山,Zhongshan,G,松山新店線,台北市中山區南京西路16號,25.052685,121.520392
68,G16,G18,南京復興,Nanjing Fuxing,G,松山新店線,台北市松山區南京東路三段253號,25.052319,121.544011
76,O05,O15,古亭,Guting,O,中和新蘆線,台北市中正區羅斯福路二段164之1號B1,25.026357,121.522873


In [8]:
geodata[geodata['station_name_tw'] == '台北車站']

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
32,R10,R13,台北車站,Taipei Main Station,R,淡水信義線,台北市中正區忠孝西路一段49號,25.046255,121.517532
109,BL12,BL7,台北車站,Taipei Main Station,BL,板南線,台北市中正區忠孝西路一段49號,25.046255,121.517532
135,A1,A1,台北車站,Taipei Main Station,A,機場線,台北市中正區鄭州路8號,25.049187,121.515245


In [9]:
geodata.drop_duplicates(subset=['station_name_tw', 'lat','lon'], keep='first', inplace=True)
geodata[geodata['station_name_tw'] == '台北車站']

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
32,R10,R13,台北車站,Taipei Main Station,R,淡水信義線,台北市中正區忠孝西路一段49號,25.046255,121.517532
135,A1,A1,台北車站,Taipei Main Station,A,機場線,台北市中正區鄭州路8號,25.049187,121.515245


#### 合併資料

In [10]:
stations = stations.merge(geodata[geodata['line_code'] != 'A'].iloc[:,2:], how='left', left_on='Station', right_on='station_name_tw', suffixes=['_left','_right'])
stations.head()

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
0,BL01,頂埔,BL,N,0,0,0,頂埔,Dingpu,BL,板南線,新北市土城區中央路四段51-6號B3,24.960120,121.420500
1,BL02,永寧,BL,N,0,0,0,永寧,Yongning,BL,板南線,新北市土城區中央路三段105號B1,24.966726,121.436072
2,BL03,土城,BL,N,0,0,0,土城,Tucheng,BL,板南線,新北市土城區金城路一段105號B1,24.973094,121.444362
3,BL04,海山,BL,N,0,0,0,海山,Haishan,BL,板南線,新北市土城區海山路39號B2,24.985339,121.448786
4,BL05,亞東醫院,BL,N,0,0,0,亞東醫院,Far Eastern Hospital,BL,板南線,新北市板橋區南雅南路二段17號B1,24.998037,121.452514


In [11]:
stations[stations['station_name_tw'] == '台北車站']

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
11,BL12,台北車站,BL,R,1,1,1,台北車站,Taipei Main Station,R,淡水信義線,台北市中正區忠孝西路一段49號,25.046255,121.517532


In [12]:
stations = stations[['Code','Station','Line','Line2','Cross_Lines','Lines_Transfer','Transportation','station_name_en','address','lat','lon']]
stations.head()

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_en,address,lat,lon
0,BL01,頂埔,BL,N,0,0,0,Dingpu,新北市土城區中央路四段51-6號B3,24.960120,121.420500
1,BL02,永寧,BL,N,0,0,0,Yongning,新北市土城區中央路三段105號B1,24.966726,121.436072
2,BL03,土城,BL,N,0,0,0,Tucheng,新北市土城區金城路一段105號B1,24.973094,121.444362
3,BL04,海山,BL,N,0,0,0,Haishan,新北市土城區海山路39號B2,24.985339,121.448786
4,BL05,亞東醫院,BL,N,0,0,0,Far Eastern Hospital,新北市板橋區南雅南路二段17號B1,24.998037,121.452514


In [13]:
stations.to_csv('./Data/Taipei_Metro_Stations_geo.csv', index=False, encoding='UTF-8')

# 7. 捷運站到站車程時間

In [2]:
stations = pd.read_csv('./Data/Taipei_Metro_Stations_geo.csv', encoding='UTF-8')
stations.head()

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_en,address,lat,lon
0,BL01,頂埔,BL,N,0,0,0,Dingpu,新北市土城區中央路四段51-6號B3,24.960120,121.420500
1,BL02,永寧,BL,N,0,0,0,Yongning,新北市土城區中央路三段105號B1,24.966726,121.436072
2,BL03,土城,BL,N,0,0,0,Tucheng,新北市土城區金城路一段105號B1,24.973094,121.444362
3,BL04,海山,BL,N,0,0,0,Haishan,新北市土城區海山路39號B2,24.985339,121.448786
4,BL05,亞東醫院,BL,N,0,0,0,Far Eastern Hospital,新北市板橋區南雅南路二段17號B1,24.998037,121.452514


In [62]:
traveltime = pd.read_csv('./Data/Taipei_Metro_Route_Traveltime_v2.csv')
traveltime.head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime
0,動物園,木柵,BR01,BR02,BR01_BR02,2
1,動物園,萬芳社區,BR01,BR03,BR01_BR03,3
2,動物園,萬芳醫院,BR01,BR04,BR01_BR04,5
3,動物園,辛亥,BR01,BR05,BR01_BR05,7
4,動物園,麟光,BR01,BR06,BR01_BR06,10


In [63]:
stations[stations['Station'] == '新北產業園區']

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_en,address,lat,lon
119,Y20,新北產業園區,Y,N,0,0,1,New Taipei Industrial Park,新北市新莊區五工路35號,25.061548,121.459926


In [64]:
def get_latitude(stationname):
    return stations[stations['Station'] == stationname]['lat'].values[0]
    
def get_longitude(stationname):
    return stations[stations['Station'] == stationname]['lon'].values[0]

In [65]:
get_latitude('新北產業園區'), get_longitude('新北產業園區')

(25.0615483, 121.4599257)

In [69]:
traveltime['From_lat'] = traveltime['StationA'].apply(lambda _ : get_latitude(_))
traveltime['From_lon'] = traveltime['StationA'].apply(lambda _ : get_longitude(_))
traveltime['To_lat'] = traveltime['StationB'].apply(lambda _ : get_latitude(_))
traveltime['To_lon'] = traveltime['StationB'].apply(lambda _ : get_longitude(_))

In [70]:
traveltime.head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime,From_lat,From_lon,To_lat,To_lon
0,動物園,木柵,BR01,BR02,BR01_BR02,2,24.998197,121.579338,24.998241,121.573145
1,動物園,萬芳社區,BR01,BR03,BR01_BR03,3,24.998197,121.579338,24.998585,121.568102
2,動物園,萬芳醫院,BR01,BR04,BR01_BR04,5,24.998197,121.579338,24.999386,121.558152
3,動物園,辛亥,BR01,BR05,BR01_BR05,7,24.998197,121.579338,25.005475,121.557107
4,動物園,麟光,BR01,BR06,BR01_BR06,10,24.998197,121.579338,25.018535,121.558791


### Google Distance Matrix API 抓取捷運站路線以汽車行駛時間

In [120]:
import googlemaps
from datetime import datetime

In [124]:
test = gmaps.distance_matrix((traveltime.at[0,'From_lat'], traveltime.at[0,'From_lon']), 
                             (traveltime.at[0,'To_lat'], traveltime.at[0,'To_lon']),
                             mode='driving')
test

{'destination_addresses': ['MRT Muzha Station, No. 135, Muzha Road, Section 4, Wenshan District, Taipei City, Taiwan 116'],
 'origin_addresses': ['No. 32, Section 2, Xinguang Road, Wenshan District, Taipei City, Taiwan 116'],
 'rows': [{'elements': [{'distance': {'text': '2.2 km', 'value': 2212},
     'duration': {'text': '6 mins', 'value': 380},
     'status': 'OK'}]}],
 'status': 'OK'}

In [125]:
def get_distance_matrix(row):
    origin = (row['From_lat'], row['From_lon'])
    destination = (row['To_lat'], row['To_lon'])
    data = gmaps.distance_matrix(origin, destination, mode='driving')['rows'][0]['elements'][0]
    drivingtime = data['duration']['value']
    distance = data['distance']['value']
    return (drivingtime, distance)

In [126]:
get_distance_matrix(traveltime.loc[0,:])

(380, 2212)

In [129]:
len(traveltime)

14042

In [134]:
drivingtime, distance = [], []

for i in trange(len(traveltime)):
    result = get_distance_matrix(traveltime.loc[i,:])
    drivingtime.append(result[0])
    distance.append(result[1])

if len(drivingtime) == len(distance):
    print(len(drivingtime))
else:
    print('兩者長度不相符')

100%|██████████| 14042/14042 [23:40<00:00,  9.88it/s] 

14042


In [137]:
traveltime['DrivingTime(s)'] = drivingtime
traveltime['Distance(m)'] = distance

In [138]:
traveltime.head()

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime,From_lat,From_lon,To_lat,To_lon,DrivingTime(s),Distance(m)
0,動物園,木柵,BR01,BR02,BR01_BR02,2,24.998197,121.579338,24.998241,121.573145,380,2212
1,動物園,萬芳社區,BR01,BR03,BR01_BR03,3,24.998197,121.579338,24.998585,121.568102,448,3130
2,動物園,萬芳醫院,BR01,BR04,BR01_BR04,5,24.998197,121.579338,24.999386,121.558152,639,4378
3,動物園,辛亥,BR01,BR05,BR01_BR05,7,24.998197,121.579338,25.005475,121.557107,708,5074
4,動物園,麟光,BR01,BR06,BR01_BR06,10,24.998197,121.579338,25.018535,121.558791,652,4925


In [140]:
traveltime[traveltime.duplicated()]

,StationA,StationB,Code_StationA,Code_StationB,From_To,TravelTime,From_lat,From_lon,To_lat,To_lon,DrivingTime(s),Distance(m)


In [141]:
traveltime.to_csv('./Data/Taipei_Metro_Route_Traveltime_v2.csv', index=False, encoding='UTF-8')

# 8. Tableau 捷運路線分時資料: 2019/6, 2020/6, 2021/6

In [2]:
from datetime import datetime

In [3]:
routes_counts = pd.read_csv('./Data/Taipei_Metro_Route_hourly_2019-202106.csv')
routes_counts.head()

,Date,Hour,Entrance,Exit,Counts
0,2019-01-01,0,松山機場,松山機場,0
1,2019-01-01,0,松山機場,中山國中,0
2,2019-01-01,0,松山機場,南京復興,0
3,2019-01-01,0,松山機場,忠孝復興,0
4,2019-01-01,0,松山機場,大安,1


In [4]:
routes_counts['Date'] = pd.to_datetime(routes_counts['Date'])
routes_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252135281 entries, 0 to 252135280
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Date      datetime64[ns]
 1   Hour      int64         
 2   Entrance  object        
 3   Exit      object        
 4   Counts    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 9.4+ GB


In [5]:
stations = pd.read_csv('./Data/Taipei_Metro_Stations_geo.csv')
stations.head()

,Code,Station,Line,Line2,Cross_Lines,Lines_Transfer,Transportation,station_name_en,address,lat,lon
0,BL01,頂埔,BL,N,0,0,0,Dingpu,新北市土城區中央路四段51-6號B3,24.960120,121.420500
1,BL02,永寧,BL,N,0,0,0,Yongning,新北市土城區中央路三段105號B1,24.966726,121.436072
2,BL03,土城,BL,N,0,0,0,Tucheng,新北市土城區金城路一段105號B1,24.973094,121.444362
3,BL04,海山,BL,N,0,0,0,Haishan,新北市土城區海山路39號B2,24.985339,121.448786
4,BL05,亞東醫院,BL,N,0,0,0,Far Eastern Hospital,新北市板橋區南雅南路二段17號B1,24.998037,121.452514


In [8]:
stationmap = {name: stations.at[i,'Code'] for i, name in stations['Station'].items()}

In [35]:
routes_counts_201906 = routes_counts[(routes_counts['Date'] >= datetime.strptime('2019-06-01','%Y-%m-%d')) & (routes_counts['Date'] < datetime.strptime('2019-06-30','%Y-%m-%d'))]
routes_counts_201906.head()

,Date,Hour,Entrance,Exit,Counts
36974880,2019-06-01,0,松山機場,松山機場,0
36974881,2019-06-01,0,松山機場,中山國中,0
36974882,2019-06-01,0,松山機場,南京復興,0
36974883,2019-06-01,0,松山機場,忠孝復興,0
36974884,2019-06-01,0,松山機場,大安,1


In [36]:
routes_counts_202006 = routes_counts[(routes_counts['Date'] >= datetime.strptime('2020-06-01','%Y-%m-%d')) & (routes_counts['Date'] < datetime.strptime('2020-06-30','%Y-%m-%d'))]
routes_counts_202006.head()

,Date,Hour,Entrance,Exit,Counts
134627303,2020-06-01,0,松山機場,松山機場,0
134627304,2020-06-01,0,松山機場,中山國中,0
134627305,2020-06-01,0,松山機場,南京復興,0
134627306,2020-06-01,0,松山機場,忠孝復興,0
134627307,2020-06-01,0,松山機場,大安,0


In [44]:
routes_counts_202106 = routes_counts[(routes_counts['Date'] >= datetime.strptime('2021-06-01','%Y-%m-%d')) & (routes_counts['Date'] < datetime.strptime('2021-06-30','%Y-%m-%d'))]
routes_counts_202106.head()

,Date,Hour,Entrance,Exit,Counts
243213851,2021-06-01,0,松山機場,松山機場,0
243213852,2021-06-01,0,松山機場,中山國中,0
243213853,2021-06-01,0,松山機場,南京復興,0
243213854,2021-06-01,0,松山機場,忠孝復興,0
243213855,2021-06-01,0,松山機場,大安,0


In [45]:
routes_counts_june = pd.concat([routes_counts_201906,routes_counts_202006,routes_counts_202106]).reset_index(drop=True)
routes_counts_june = routes_counts_june[routes_counts_june['Entrance'] != routes_counts_june['Exit']]
routes_counts_june['From_To'] = routes_counts_june['Entrance'].map(stationmap) + '_' + routes_counts_june['Exit'].map(stationmap)
routes_counts_june.head()

,Date,Hour,Entrance,Exit,Counts,From_To
1,2019-06-01,0,松山機場,中山國中,0,BR13_BR12
2,2019-06-01,0,松山機場,南京復興,0,BR13_BR11
3,2019-06-01,0,松山機場,忠孝復興,0,BR13_BL15
4,2019-06-01,0,松山機場,大安,1,BR13_BR09
5,2019-06-01,0,松山機場,科技大樓,1,BR13_BR08


In [48]:
routes_counts_june = pd.melt(routes_counts_june, id_vars=['Date','Hour','From_To','Counts'], value_vars=['Entrance','Exit'], var_name='Point_Order', value_name='Station')

In [50]:
routes_counts_june['Point_Order'] = routes_counts_june['Point_Order'].apply(lambda _: 1 if _ == 'Entrance' else 2 )
routes_counts_june.head()

,Date,Hour,From_To,Counts,Point_Order,Station
0,2019-06-01,0,BR13_BR12,0,1,松山機場
1,2019-06-01,0,BR13_BR11,0,1,松山機場
2,2019-06-01,0,BR13_BL15,0,1,松山機場
3,2019-06-01,0,BR13_BR09,1,1,松山機場
4,2019-06-01,0,BR13_BR08,1,1,松山機場


In [51]:
routes_counts_june.to_csv('./Data/Taipei_Metro_Route_hourly_201906-202006-202106.csv', index=False)